# Day 7

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])
print('x_train ' ,x_train.shape)
print('y_train ', y_train.shape)

x_train  torch.Size([8, 3])
y_train  torch.Size([8])


In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])
print('x_test ' ,x_test.shape)
print('y_test ', y_test.shape)

x_test  torch.Size([3, 3])
y_test  torch.Size([3])


In [5]:
class softmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,3)

    def forward(self,x):
        return self.linear(x)
    
   

In [6]:
model = softmaxClassifierModel()

In [7]:
# Optimizer
optimizer = optim.SGD(model.parameters(), lr= 0.1)

In [8]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x)
        prediction = model(x_train)

        # cost
        cost = F.cross_entropy(prediction, y_train)

        # cost로 H(x)개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [9]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(
        correct_count / len(y_test) *100, cost.item()
    ))

In [10]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [11]:
# 이미 오버피팅된것
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


## Learning rate
learning rate가 너무 크면 diverge 하면서 cost가 점점 늘어난다(overshooting).<br>
learning rate가 너무 작으면 cost가 거의 줄어들지 않는다.<br>
적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.<br>

In [44]:
model = softmaxClassifierModel()

In [45]:
optimizer = optim.SGD(model.parameters(), lr = 1e-1)

In [46]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.939317
Epoch    1/20 Cost: 1.887239
Epoch    2/20 Cost: 1.055398
Epoch    3/20 Cost: 0.936401
Epoch    4/20 Cost: 0.917493
Epoch    5/20 Cost: 0.911811
Epoch    6/20 Cost: 0.906384
Epoch    7/20 Cost: 0.901102
Epoch    8/20 Cost: 0.895959
Epoch    9/20 Cost: 0.890947
Epoch   10/20 Cost: 0.886062
Epoch   11/20 Cost: 0.881298
Epoch   12/20 Cost: 0.876650
Epoch   13/20 Cost: 0.872114
Epoch   14/20 Cost: 0.867685
Epoch   15/20 Cost: 0.863359
Epoch   16/20 Cost: 0.859132
Epoch   17/20 Cost: 0.855000
Epoch   18/20 Cost: 0.850961
Epoch   19/20 Cost: 0.847010


## Data Preprocessing

In [47]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 값을 보니 regression 문제인듯

In [48]:
# 정규화
mu = x_train.mean(dim = 0)

In [49]:
sigma = x_train.std(dim = 0)

In [50]:
norm_x_train = (x_train - mu) / sigma

In [52]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [53]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 3 개의 입력을 받아 하나의 결과를 뱉는 layre

    def forward(self, x):
        return self.linear(x)

In [54]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [55]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [56]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29474.621094
Epoch    1/20 Cost: 18722.042969
Epoch    2/20 Cost: 11941.123047
Epoch    3/20 Cost: 7630.645996
Epoch    4/20 Cost: 4880.464844
Epoch    5/20 Cost: 3122.821289
Epoch    6/20 Cost: 1998.640259
Epoch    7/20 Cost: 1279.364014
Epoch    8/20 Cost: 819.077271
Epoch    9/20 Cost: 524.501465
Epoch   10/20 Cost: 335.968170
Epoch   11/20 Cost: 215.298920
Epoch   12/20 Cost: 138.062103
Epoch   13/20 Cost: 88.621628
Epoch   14/20 Cost: 56.971203
Epoch   15/20 Cost: 36.706795
Epoch   16/20 Cost: 23.729731
Epoch   17/20 Cost: 15.416880
Epoch   18/20 Cost: 10.089418
Epoch   19/20 Cost: 6.672885
